In [1]:
 # Use termcolor to make it easy to colorize the outputs.
from IPython import get_ipython
from IPython.display import display
# %%
 # Use termcolor to make it easy to colorize the outputs.
!pip install termcolor > /dev/null
!pip install langchain
!pip install openai
!pip install langchain_experimental
!pip install tiktoken
!pip install faiss-cpu==1.7.4
!pip install pydantic==2.9.2
from datetime import datetime, timedelta
from typing import List
import math
import faiss
import os
import logging
logging.basicConfig(level=logging.ERROR)
from langchain.cache import InMemoryCache # Import InMemoryCache
from langchain_experimental.generative_agents import GenerativeAgentMemory
from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS
from termcolor import colored
from langchain_experimental.generative_agents import (

    GenerativeAgent,
    GenerativeAgentMemory,
)

In [13]:
os.environ["OPENAI_API_KEY"] = 'API KEY HERE'

In [14]:
USER_NAME = "Buddy"  # The name you want to use when interviewing the agent.

LLM = ChatOpenAI(max_tokens=1500)  # Can be any LLM you want.

## Implementing Your First Generative Agent




In [15]:
def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OpenAIEmbeddings()
    # Initialize the vectorstore as empty
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    # Remove relevance_score_fn or define it before this line
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        # relevance_score_fn=relevance_score_fn,  # Remove this line if not needed
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

In [16]:
from langchain_community.cache import BaseCache, InMemoryCache # Import BaseCache from langchain_community

cache = InMemoryCache()

alexis_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works,
    cache=cache # Pass the cache here
)

# Define Alexis

alexis = GenerativeAgent(
    name="Alexis",
    age=21,
    traits="curious, helpful",
    status="looking for a challenge",
    memory_retriever=alexis_memory.memory_retriever,
    llm=LLM,
    memory=alexis_memory,
)

# After initializing alexis_memory, call model_rebuild
alexis_memory.model_rebuild()



In [17]:
# The current "Summary" of a character can't be made because the agent hasn't made
# any observations yet.
print(alexis.get_summary())

Name: Alexis (age: 21)
Innate traits: curious, helpful
Alexis is a hardworking and dedicated individual who values honesty and integrity. She is confident in her abilities and takes pride in her work. Additionally, Alexis is a team player who is always willing to help others and collaborate on projects.


In [18]:
# We can add memories directly to the memory object

alexis_observations = [
    "Alexis recalls her morning walk in the park",
    "Alexis feels excited about the new book she started reading",
    "Alexis remembers her conversation with a close friend",
    "Alexis thinks about the painting she saw at the art gallery",
    "Alexis is planning to learn a new recipe for dinner",
    "Alexis is looking forward to her weekend trip",
    "Alexis contemplates her goals for the month."
]

for observation in alexis_observations:
    alexis.memory.add_memory(observation)



# We will see how this summary updates after more observations to create a more rich description.
print(alexis.get_summary(force_refresh=True))

Name: Alexis (age: 21)
Innate traits: curious, helpful
Alexis is reflective, goal-oriented, adventurous, appreciative of nature and art, and enjoys learning and trying new things.


## Interacting and Providing Context to Generative Characters

## Pre-Interview with Character

Before sending our character on their way, let's ask them a few questions.

In [19]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [20]:
interview_agent(alexis, "What do you like to do?")


'Alexis said "I enjoy reading, trying new recipes, going on adventures, and appreciating nature and art. How about you, Buddy?"'

## Step through the day's observations.

In [21]:
# Let's give Alexis a series of observations to reflect on her day
# Adding observations to Alexis' memory
alexis_observations_day = [
    "Alexis starts her day with a refreshing yoga session.",
    "Alexis spends time writing in her journal.",
    "Alexis experiments with a new recipe she found online.",
    "Alexis gets lost in her thoughts while gardening.",
    "Alexis decides to call her grandmother for a heartfelt chat.",
    "Alexis relaxes in the evening by playing her favorite piano pieces.",
]

for observation in alexis_observations_day:
    alexis.memory.add_memory(observation)


In [22]:
# Let's observe how Alexis's day influences her memory and character
for i, observation in enumerate(alexis_observations_day):
    _, reaction = alexis.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i + 1) % len(alexis_observations_day)) == 0:
        print("*" * 40)
        print(
            colored(
                f"After these observations, Alexis's summary is:\n{alexis.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)


Alexis starts her day with a refreshing yoga session. Alexis smiles and feels grateful for starting her day on a positive note.
Alexis spends time writing in her journal. Alexis feels a sense of calm and satisfaction as she reflects on her thoughts and experiences in her journal.
Alexis experiments with a new recipe she found online. Alexis feels proud of herself for trying something new and looks forward to tasting the finished dish.
Alexis gets lost in her thoughts while gardening. Alexis feels a sense of peace and connection with nature as she loses herself in her thoughts while gardening.
Alexis decides to call her grandmother for a heartfelt chat. Alexis smiles and feels grateful for the opportunity to connect with her grandmother.
Alexis relaxes in the evening by playing her favorite piano pieces. Alexis feels a sense of peace and joy as she plays her favorite piano pieces.
****************************************
After these observations, Alexis's summary is:
Name: Alexis (age: 

# Adding Multiple Characters